Source: 
https://stackoverflow.com/questions/47585723/kmeans-clustering-in-pyspark

In [2]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
# $example off$

from pyspark.sql import SparkSession
import os
import sys
from pyspark.context import SparkContext,SparkConf
from pyspark.rdd import RDD
!pip install psutil
import psutil
#若沒有下面這段
#會有Exception: Java gateway process exited before sending its port number
#Config
os.environ['JAVA_HOME'] = 'C:\Program Files\Java\jre1.8.0_301'
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable


sc = SparkContext(conf=SparkConf().setAppName("Kmeans").setMaster("local"))
print(f'sc:{sc}')
spark = SparkSession(sc)

You should consider upgrading via the 'c:\users\leoshr\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


sc:<SparkContext master=local appName=Kmeans>


In [2]:
spark.version

'3.1.2'

In [7]:
os.getcwd()

'C:\\Users\\LeoShr\\p_space\\NTHU\\MDA\\CH3_K_means'

In [8]:
#os.chdir('C:\\Users\\LeoShr\\p_space\\NTHU\\MDA\\CH3_K_means') #change the current working directory to specified path. 
os.listdir()


['.ipynb_checkpoints',
 '1106Trial_利用RDD.ipynb',
 '1106Trial_試圖用df做失敗.ipynb',
 '1107df_很好的stackoverflow文章 成功.ipynb',
 '1107df_很好的stackoverflow文章_模擬資料.ipynb',
 '1107Spark ml_df官方範例.ipynb',
 'Apache_PySpark_Non-custom_Kmeans_1.ipynb',
 'Apache_PySpark_Non-custom_Kmeans_2.ipynb',
 'Credit Card Dataset for Clustering_K_means_成功.ipynb',
 'dataset',
 'K-means clustering_hacker_pyspark__mocking.ipynb',
 'kmeans',
 'KMeans function.ipynb',
 'MDA_HW3.pdf',
 'Project',
 'target',
 'testing5_.ipynb',
 'Trail_3_testing.ipynb']

In [3]:
df = spark.createDataFrame([[0, 33.3, -17.5],
                              [1, 40.4, -20.5],
                              [2, 28., -23.9],
                              [3, 29.5, -19.0],
                              [4, 32.8, -18.84]
                             ],
                              ["other","lat", "long"])

In [4]:
df

DataFrame[other: bigint, lat: double, long: double]

In [5]:
df.show()


+-----+----+------+
|other| lat|  long|
+-----+----+------+
|    0|33.3| -17.5|
|    1|40.4| -20.5|
|    2|28.0| -23.9|
|    3|29.5| -19.0|
|    4|32.8|-18.84|
+-----+----+------+



### Step 1 - assemble your features

In contrast to most ML packages out there, Spark ML requires your input features to be gathered in a single column of your dataframe, usually named features; and it provides a specific method for doing this, VectorAssembler:

In [6]:
from pyspark.ml.feature import VectorAssembler

vecAssembler = VectorAssembler(inputCols=["lat", "long"], outputCol="features")
new_df = vecAssembler.transform(df)
new_df.show()

# As perhaps already guessed, the argument inputCols serves to tell VectoeAssembler which particular columns 
# in our dataframe are to be used as features.

+-----+----+------+-------------+
|other| lat|  long|     features|
+-----+----+------+-------------+
|    0|33.3| -17.5| [33.3,-17.5]|
|    1|40.4| -20.5| [40.4,-20.5]|
|    2|28.0| -23.9| [28.0,-23.9]|
|    3|29.5| -19.0| [29.5,-19.0]|
|    4|32.8|-18.84|[32.8,-18.84]|
+-----+----+------+-------------+



## Step 2 - fit your KMeans model

In [7]:
from pyspark.ml.clustering import KMeans

kmeans = KMeans(k=2, seed=1)  # 2 clusters here
model = kmeans.fit(new_df.select('features'))

# select('features') here serves to tell the algorithm 
# which column of the dataframe to use for clustering - remember that, after Step 1 above, your original lat & long features 
# are no more directly used.

### Step 3 - transform your initial dataframe to include cluster assignments



In [8]:
transformed = model.transform(new_df)
transformed.show()    

+-----+----+------+-------------+----------+
|other| lat|  long|     features|prediction|
+-----+----+------+-------------+----------+
|    0|33.3| -17.5| [33.3,-17.5]|         0|
|    1|40.4| -20.5| [40.4,-20.5]|         0|
|    2|28.0| -23.9| [28.0,-23.9]|         1|
|    3|29.5| -19.0| [29.5,-19.0]|         1|
|    4|32.8|-18.84|[32.8,-18.84]|         0|
+-----+----+------+-------------+----------+



In [ ]:
# The last column of the transformed dataframe, prediction, shows the cluster assignment - in my toy case, I have ended up with 4 records in cluster #0 and 1 record in cluster #1.

# You can further manipulate the transformed dataframe with select statements, or even drop the features column (which has now fulfilled its function and may be no longer necessary)...

# Hopefully you are much closer now to what you actually wanted to achieve in the first place. For extracting cluster statistics etc., another recent answer of mine might be helpful...

In [9]:
#延伸: https://stackoverflow.com/questions/47130299/pyspark-ml-get-kmeans-cluster-statistics/47156822#47156822

In [6]:
from pyspark.sql import SQLContext
import dataset
load(iris)

NameError: name 'load' is not defined

In [ ]:
data = spark.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load('iris.csv')
data.show()